In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Dataset
import cv2
import numpy as np
import os
from PIL import Image

In [2]:
# Custom dataset for CASIA v2 with preprocessing
class CASIADataset(Dataset):
    def __init__(self, root_dir, transform=None, apply_fft=False):
        self.root_dir = root_dir
        self.transform = transform
        self.apply_fft = apply_fft
        self.images = []
        self.labels = []
        
        for label, category in enumerate(['authentic', 'tampered']):
            category_path = os.path.join(root_dir, category)
            for img_name in os.listdir(category_path):
                img_path = os.path.join(category_path, img_name)
                self.images.append(img_path)
                self.labels.append(label)
    
    def preprocess_fft(self, image):
        gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)
        f = np.fft.fft2(gray)
        fshift = np.fft.fftshift(f)
        magnitude_spectrum = 20 * np.log(np.abs(fshift) + 1e-8)
        
        # Convert grayscale FFT output to 3-channel
        magnitude_spectrum = np.stack([magnitude_spectrum] * 3, axis=-1)  

        return Image.fromarray(magnitude_spectrum.astype(np.uint8))
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        
        if self.apply_fft:
            image = self.preprocess_fft(image)
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [3]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [4]:
# Load dataset
train_dataset = CASIADataset(root_dir="dataset/train", transform=transform, apply_fft=True)
val_dataset = CASIADataset(root_dir="dataset/val", transform=transform, apply_fft=True)
test_dataset = CASIADataset(root_dir="dataset/test", transform=transform, apply_fft=True)

In [5]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [6]:
# Define CNN model
class ForgeryCNN(nn.Module):
    def __init__(self):
        super(ForgeryCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 28 * 28, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

In [7]:
# Initialize model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ForgeryCNN().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
# Training loop
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        model.train()
        total_loss, correct = 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device, dtype=torch.float32)
            optimizer.zero_grad()
            outputs = model(images).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            correct += ((outputs > 0.5).float() == labels).sum().item()
        
        val_loss, val_correct = 0, 0
        model.eval()
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device, dtype=torch.float32)
                outputs = model(images).squeeze()
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                val_correct += ((outputs > 0.5).float() == labels).sum().item()
        
        print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss/len(train_loader):.4f}, Accuracy: {correct/len(train_loader.dataset):.4f}, Val Loss: {val_loss/len(val_loader):.4f}, Val Acc: {val_correct/len(val_loader.dataset):.4f}")



In [9]:
# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10)

Epoch 1/10 - Loss: 0.6150, Accuracy: 0.7069, Val Loss: 0.6006, Val Acc: 0.7086
Epoch 2/10 - Loss: 0.5963, Accuracy: 0.7079, Val Loss: 0.5851, Val Acc: 0.7076
Epoch 3/10 - Loss: 0.5833, Accuracy: 0.7087, Val Loss: 0.5743, Val Acc: 0.7105
Epoch 4/10 - Loss: 0.5780, Accuracy: 0.7100, Val Loss: 0.5720, Val Acc: 0.7171
Epoch 5/10 - Loss: 0.5733, Accuracy: 0.7075, Val Loss: 0.5548, Val Acc: 0.7267
Epoch 6/10 - Loss: 0.5711, Accuracy: 0.7113, Val Loss: 0.5554, Val Acc: 0.7248
Epoch 7/10 - Loss: 0.5734, Accuracy: 0.7073, Val Loss: 0.5673, Val Acc: 0.7181
Epoch 8/10 - Loss: 0.5673, Accuracy: 0.7134, Val Loss: 0.5588, Val Acc: 0.7210
Epoch 9/10 - Loss: 0.5583, Accuracy: 0.7197, Val Loss: 0.5686, Val Acc: 0.7324
Epoch 10/10 - Loss: 0.5385, Accuracy: 0.7253, Val Loss: 0.5599, Val Acc: 0.7219


In [10]:
# Test the model
def test_model(model, test_loader):
    model.eval()
    test_correct = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device, dtype=torch.float32)
            outputs = model(images).squeeze()
            test_correct += ((outputs > 0.5).float() == labels).sum().item()
    
    print(f"Test Accuracy: {test_correct / len(test_loader.dataset):.4f}")


In [11]:
# Run model on test data
test_model(model, test_loader)

Test Accuracy: 0.7157


In [13]:
torch.save(model.state_dict(), "model.pth")